# Chapter2
D3.js in Actionの2章の勉強ノートです。



In [1]:
%%html
<script src="js/d3.v3.min.js" charset="utf-8"></script>

## データの読み込み
D3では様々なデータをサポートしています。
- TEXT: d3.text()
- XML: d3.xml()
- CSV: d3.csv()
- JSON: d3.json()
- HTML: d3.html()

pythonとのインタフェースを取ることを考えると、一般的に構造を保持できるJSONとCSVがデータの受け渡しに使われるます。

例として以下のようなcities.csvを読み込んでみましょう。

In [7]:
!cat data/cities.csv

"label","population","country","x","y"
"San Francisco", 750000,"USA",37,-122
"Fresno", 500000,"USA",36,-119
"Lahore",12500000,"Pakistan",31,74
"Karachi",13000000,"Pakistan",24,67
"Rome",2500000,"Italy",41,12
"Naples",1000000,"Italy",40,14
"Rio",12300000,"Brazil",-22,-43
"Sao Paolo",12300000,"Brazil",-23,-46

読み込まれたデータは、function(error, data)形式のコールバックで与えられます。
このコールバックの中で実行したい処理を記述する方式になります。

In [8]:
%%javascript
d3.csv("data/cities.csv",function(error,data) {console.log(error,data)});

<IPython.core.display.Javascript object>

javascriptのコンソールに以下のようにデータの内容が出力されます。

![cities console log](images/cities_log.png)

これを見るとCSVのデータがヘッダのカラム名をキーとする辞書の配列として渡されていることがわかります。

## Pythonデータの受け渡し
jupyterの計算結果をD3に渡す方法を以下に紹介します。

jsonとTemplateライブラリを使用します。


In [28]:
from IPython.core.display import HTML
from string import Template
import json
import pandas as pd


pandasを使ってcities.csvを読み込み、データフレームdfにセットします。

In [29]:
df = pd.read_csv("data/cities.csv")
df.head()

,label,population,country,x,y
0,San Francisco,750000,USA,37,-122
1,Fresno,500000,USA,36,-119
2,Lahore,12500000,Pakistan,31,74
3,Karachi,13000000,Pakistan,24,67
4,Rome,2500000,Italy,41,12


Templateを使ってscriptタグにdfを変数dataに代入します。

$python_dataの置換で、json.dumpsとto_dict(orient='records')を使用するのがポイントです。


In [36]:
data_text = Template('''
<script>
    var data = $python_data ;
    console.log(data);
</script>
''')
data_text = data_text.substitute({'python_data': json.dumps(df.to_dict(orient='records'))})

In [37]:
data_text

'\n<script>\n    var data = [{"y": -122, "country": "USA", "population": 750000, "x": 37, "label": "San Francisco"}, {"y": -119, "country": "USA", "population": 500000, "x": 36, "label": "Fresno"}, {"y": 74, "country": "Pakistan", "population": 12500000, "x": 31, "label": "Lahore"}, {"y": 67, "country": "Pakistan", "population": 13000000, "x": 24, "label": "Karachi"}, {"y": 12, "country": "Italy", "population": 2500000, "x": 41, "label": "Rome"}, {"y": 14, "country": "Italy", "population": 1000000, "x": 40, "label": "Naples"}, {"y": -43, "country": "Brazil", "population": 12300000, "x": -22, "label": "Rio"}, {"y": -46, "country": "Brazil", "population": 12300000, "x": -23, "label": "Sao Paolo"}] ;\n    console.log(data);\n</script>\n'

以下のコマンドを実行して、javascriptのコンソールをみてください。d3で読み込んだ時と同じログが出力されています。

In [39]:
HTML(data_text)

## スケールマッピング
D3のスケール処理はとても良くできており、データに応じて選べるようになっています。
- d3.scale.linear(): 線形補間
- d3.scale.quantile(): 分位数（区間で分けられた値）

最初に、線形補間を見てみましょう。
domain関数では、問題領域（ここではデータの分布領域）をrange関数で指定された範囲にマッピングします。

console.logの代わりにelement.textを使うとjupyterのノートブックに出力できます（ただし１回だけ指定可能みたい）。

In [58]:
%%javascript
var newRamp = d3.scale.linear().domain([500000,13000000]).range([0, 500]);
element.text(newRamp(1000000) + ", " +newRamp(9000000)+", "+newRamp.invert(313));

<IPython.core.display.Javascript object>

D３のscaleの凄いのは、数値だけでなくカラーにもマッピングできることです。

In [57]:
%%javascript
var newRamp = d3.scale.linear().domain([500000,13000000]).range(["blue",
"red"]);
element.text(newRamp(1000000) + ", " +newRamp(9000000));

<IPython.core.display.Javascript object>

### カテゴリわけ
次にquantile関数を使っていくつかのカテゴリに分けてみます。

以下の例では、sampleArrayのデータを３つのグループに分けて、small, medium, largetとします。
値ではなく、ソートした後にデータ数が等分になるようにカテゴリわけしているみたいです。

[1,10,44], [58,66,124], [423, 524, 900]から

(-∞..44], (44..124], (124..∞)の区間をsmall, medium, largeにマッピングしています。

In [65]:
%%javascript
var sampleArray = [423,124,66,424,58,10,900,44,1];
var qScaleName =
d3.scale.quantile().domain(sampleArray).range(["small","medium","large"]);
element.text(qScaleName(68) + ", " +qScaleName(20)+", "+qScaleName(10000));

<IPython.core.display.Javascript object>